# Atividade 6
## Grupo 3

In [43]:
import pandas as pd
from scipy.stats import shapiro, ttest_ind

df = pd.read_excel('2025 2ºsem M109 Cell Phone Dependence Scale.xlsx')
df.columns = df.columns.str.strip()
df.head()

,ID,Sexo:,Idade (anos):,Peso (kg):,Altura (m):,Coluna1,Curso:,Período:,Turno de estudo:,Faz alguma atividade extraclasse. se sim qual (is)?,...,13- Com que frequência você se mantém conectado ao celular quando está com amigos ou família?,Pontos 13,14- Com que frequência você se sente sozinho se não participa de redes sociais ou quando não está em grupo enquanto outros estão?,Pontos 14,15- Quantas vezes você mantém na agenda do celular o número de um médico. psicólogo ou hospital por medo de se sentir mal na rua?,Pontos 15,16- Com que frequência você toca no celular?,Pontos 16,Pontuação final,Classificação
0,2,Masculino,18,73.0,1.84,21.561909,Engenharia de Computação,P3,Integral,Iniciação Científica ;,...,Frequentemente,1,Nunca / Raramente,0,Frequentemente,1,Sempre,2,NaN,NaN
1,3,Masculino,18,75.0,1.75,24.489796,Engenharia de Software,P3,Integral,Outros;,...,Frequentemente,1,Frequentemente,1,Sempre,2,Sempre,2,NaN,NaN
2,4,Feminino,22,67.0,1.73,22.386314,Engenharia de Software,P9,Noturno,Monitoria ;Estágio ;,...,Frequentemente,1,Nunca / Raramente,0,Frequentemente,1,Nunca / Raramente,0,NaN,NaN
3,5,Masculino,20,85.0,1.82,25.661152,Engenharia de Computação,P6,Integral,Trabalho ;,...,Nunca / Raramente,0,Nunca / Raramente,0,Nunca / Raramente,0,Frequentemente,1,NaN,NaN
4,6,Masculino,19,64.0,1.90,17.728532,Engenharia de Telecomunicações,P5,Integral,Iniciação Científica ;,...,Nunca / Raramente,0,Nunca / Raramente,0,Nunca / Raramente,0,Frequentemente,1,NaN,NaN


In [44]:
colunas_pontos = [f"Pontos {i}" for i in range(1, 17)]
df["Pontuação final"] = df[colunas_pontos].sum(axis=1)

def classificar(pontuacao):
    if pontuacao <= 10:
        return "Leve"
    elif pontuacao <= 20:
        return "Moderado"
    else:
        return "Grave"
    
df["Classificação"] = df["Pontuação final"].apply(classificar)

### 1) Testar normalidade dos dados

In [45]:
from scipy.stats import shapiro

grupo_sozinho = df[df["Com quem mora:"].str.contains("sozinh", case=False, na=False)]["Pontuação final"]
grupo_outros = df[~df["Com quem mora:"].str.contains("sozinh", case=False, na=False)]["Pontuação final"]

stat_sozinho, p_sozinho = shapiro(grupo_sozinho)
stat_outros, p_outros = shapiro(grupo_outros)

print("Teste de Normalidade (Shapiro–Wilk)")
print("---------------------------------------")
print(f"Mora sozinho       → W = {stat_sozinho:.4f}, p = {p_sozinho:.4f}")
print(f"Mora acompanhado   → W = {stat_outros:.4f}, p = {p_outros:.4f}")

alpha = 0.05
print("\nInterpretação:")
print(f"Se p > {alpha}, não rejeitamos H0 (distribuição normal).")
print(f"Se p ≤ {alpha}, rejeitamos H0 (distribuição não normal).")


Teste de Normalidade (Shapiro–Wilk)
---------------------------------------
Mora sozinho       → W = 0.9647, p = 0.0456
Mora acompanhado   → W = 0.9902, p = 0.3664

Interpretação:
Se p > 0.05, não rejeitamos H0 (distribuição normal).
Se p ≤ 0.05, rejeitamos H0 (distribuição não normal).


### 2) Avaliar homoscedasticidade dos dados

In [46]:
from scipy.stats import levene

stat_levene, p_levene = levene(grupo_sozinho, grupo_outros, center='median')

print("Teste de Homoscedasticidade (Levene)")
print("---------------------------------------")
print(f"Estatística W = {stat_levene:.4f}, p = {p_levene:.4f}")

alpha = 0.05
print("\nInterpretação:")
if p_levene > alpha:
    print(f"p = {p_levene:.4f} > {alpha} → Não rejeitamos H0 → Variâncias iguais (homoscedásticas).")
else:
    print(f"p = {p_levene:.4f} ≤ {alpha} → Rejeitamos H0 → Variâncias diferentes (heteroscedásticas).")


Teste de Homoscedasticidade (Levene)
---------------------------------------
Estatística W = 3.4128, p = 0.0660

Interpretação:
p = 0.0660 > 0.05 → Não rejeitamos H0 → Variâncias iguais (homoscedásticas).


### 3) Comparar as médias entre os grupos

In [47]:
from scipy.stats import ttest_ind

stat_t, p_t = ttest_ind(grupo_sozinho, grupo_outros, equal_var=True)

media_sozinho = grupo_sozinho.mean()
media_outros = grupo_outros.mean()
std_sozinho = grupo_sozinho.std()
std_outros = grupo_outros.std()

print("Teste t de Student (amostras independentes)")
print("----------------------------------------------")
print(f"Mora sozinho       → média = {media_sozinho:.2f}, desvio = {std_sozinho:.2f}, n = {len(grupo_sozinho)}")
print(f"Mora acompanhado   → média = {media_outros:.2f}, desvio = {std_outros:.2f}, n = {len(grupo_outros)}")
print(f"\nEstatística t = {stat_t:.4f}, p = {p_t:.4f}")

alpha = 0.05
print("\nInterpretação:")
if p_t > alpha:
    print(f"p = {p_t:.4f} > {alpha} → Não rejeitamos H0 → Não há diferença significativa entre as médias.")
else:
    print(f"p = {p_t:.4f} ≤ {alpha} → Rejeitamos H0 → Há diferença significativa entre as médias.")


Teste t de Student (amostras independentes)
----------------------------------------------
Mora sozinho       → média = 14.70, desvio = 4.95, n = 70
Mora acompanhado   → média = 15.67, desvio = 5.69, n = 154

Estatística t = -1.2279, p = 0.2208

Interpretação:
p = 0.2208 > 0.05 → Não rejeitamos H0 → Não há diferença significativa entre as médias.


### 4) Formular as hipóteses para cada teste (Normalidade, Variância e Médias)

#### Teste de Normalidade (Shapiro–Wilk)
Aplicado separadamente para cada grupo, com nível de significância α = 0,05.

- **Grupo “Mora sozinho”**  
  - H₀: A variável *Pontuação final* segue uma distribuição normal.  
  - H₁: A variável *Pontuação final* não segue uma distribuição normal.

- **Grupo “Mora com outras pessoas”**  
  - H₀: A variável *Pontuação final* segue uma distribuição normal.  
  - H₁: A variável *Pontuação final* não segue uma distribuição normal.

---

#### Teste de Homoscedasticidade (Levene)
Verifica se as variâncias populacionais dos grupos são iguais (robusto a não-normalidade).

- H₀: As variâncias populacionais são iguais (σ₁² = σ₂²).  
- H₁: As variâncias populacionais são diferentes (σ₁² ≠ σ₂²).

---

#### Teste de Comparação de Médias
O teste será escolhido conforme a normalidade:
- Se ambos os grupos forem normais → **t de Student (amostras independentes)**  
- Se ao menos um grupo não for normal → **teste de Mann–Whitney U (não paramétrico)**

Hipóteses gerais:
- H₀: As médias populacionais são iguais (μ₁ = μ₂).  
- H₁: As médias populacionais são diferentes (μ₁ ≠ μ₂).


### 5) Faça print dos cálculos realizados.

In [48]:
print("Resumo dos Testes Estatísticos")
print("=================================")
print(f"Shapiro–Wilk (sozinho): p = {p_sozinho:.4f}")
print(f"Shapiro–Wilk (acompanhado): p = {p_outros:.4f}")
print(f"Levene: p = {p_levene:.4f}")
print(f"t de Student: p = {p_t:.4f}")

print("\nResumo:")
print("---------------------------------")
print("Normalidade: Mora sozinho não normal (p < 0.05); Mora acompanhado normal (p > 0.05).")
print("Variâncias: homogêneas (p_Levene > 0.05).")
print("Médias: não há diferença significativa (p_t > 0.05).")


Resumo dos Testes Estatísticos
Shapiro–Wilk (sozinho): p = 0.0456
Shapiro–Wilk (acompanhado): p = 0.3664
Levene: p = 0.0660
t de Student: p = 0.2208

Resumo:
---------------------------------
Normalidade: Mora sozinho não normal (p < 0.05); Mora acompanhado normal (p > 0.05).
Variâncias: homogêneas (p_Levene > 0.05).
Médias: não há diferença significativa (p_t > 0.05).


### 6) Coloque a decisão Estatística

| Teste | Estatística | Valor p | α | Decisão |
|--------|--------------|---------|------|-----------|
| Shapiro–Wilk (Mora sozinho) | 0.9647 | 0.0456 | 0.05 | Rejeitar H₀ — distribuição não normal |
| Shapiro–Wilk (Mora acompanhado) | 0.9902 | 0.3664 | 0.05 | Não rejeitar H₀ — distribuição normal |
| Levene | 3.4128 | 0.0660 | 0.05 | Não rejeitar H₀ — variâncias iguais |
| t de Student | — | 0.2208 | 0.05 | Não rejeitar H₀ — médias iguais |

### 7) Elabore uma conclusão

Os testes estatísticos para o **Grupo 3 (Mora sozinho × Mora com outras pessoas)** indicam que:

- O grupo “Mora sozinho” **não apresentou distribuição normal** (p = 0,0456), enquanto o grupo “Mora com outras pessoas” apresentou (p = 0,3664).  
- As **variâncias são homogêneas** entre os grupos (p = 0,0660).  
- O **teste t de Student** indicou **p = 0,2208 > 0,05**, logo **não há diferença significativa** entre as médias.

**Conclusão final:**  
O fato de o estudante **morar sozinho ou com outras pessoas** **não influencia significativamente** o nível médio de dependência em smartphones.  
A hipótese nula de igualdade das médias é **mantida**, indicando que o tipo de moradia **não é um fator determinante** para a dependência.


In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib import colors

# === Caminho do PDF de saída ===
output_path = "Atividade6_Grupo3.pdf"

# === Criação do documento ===
doc = SimpleDocTemplate(output_path, pagesize=A4)
story = []
styles = getSampleStyleSheet()

# === Estilos customizados ===
title_style = ParagraphStyle("title", parent=styles["Heading1"], alignment=1, fontSize=16, spaceAfter=20)
subtitle_style = ParagraphStyle("subtitle", parent=styles["Heading2"], spaceAfter=10)
text_style = ParagraphStyle("text", parent=styles["BodyText"], leading=15, fontSize=11)

# === Capa ===
story.append(Paragraph("Instituto Nacional de Telecomunicações (INATEL)", title_style))
story.append(Paragraph("Disciplina: M109 - Estatística", text_style))
story.append(Spacer(1, 20))
story.append(Paragraph("<b>Atividade 6 – Testes Paramétricos</b>", subtitle_style))
story.append(Paragraph("Variável analisada: Dependência em smartphones", text_style))
story.append(Paragraph("Grupo 3 – Mora sozinho × Mora com outras pessoas", text_style))
story.append(Spacer(1, 20))

# === Seções conforme o PDF original ===
sections = [
    ("1) Testar normalidade dos dados", 
     f"Shapiro–Wilk:\n\nMora sozinho → p = {p_sozinho:.4f}\nMora acompanhado → p = {p_outros:.4f}\n\n"
     "Interpretação: o grupo 'Mora sozinho' não apresenta distribuição normal (p < 0,05), "
     "enquanto 'Mora com outras pessoas' apresenta normalidade (p > 0,05)."),
    
    ("2) Avaliar homoscedasticidade dos dados",
     f"Levene (center='median'):\n\np = {p_levene:.4f}\n\n"
     "Interpretação: p > 0,05 → não rejeitamos H0, as variâncias são homogêneas."),
    
    ("3) Comparar as médias entre os grupos",
     f"t de Student (equal_var=True):\n\np = {p_t:.4f}\n\n"
     "Interpretação: p > 0,05 → não rejeitamos H0, não há diferença significativa entre as médias."),
    
    ("4) Formular as hipóteses",
     "<b>Normalidade:</b> H0: distribuição normal / H1: não normal.<br/>"
     "<b>Variância:</b> H0: variâncias iguais / H1: diferentes.<br/>"
     "<b>Médias:</b> H0: médias iguais / H1: diferentes."),
    
    ("5) Print dos cálculos realizados",
     f"Resultados obtidos no notebook:\n\n"
     f"• Shapiro (sozinho): {p_sozinho:.4f}\n"
     f"• Shapiro (acompanhado): {p_outros:.4f}\n"
     f"• Levene: {p_levene:.4f}\n"
     f"• t de Student: {p_t:.4f}"),
    
    ("6) Decisão estatística",
     "• Normalidade: apenas o grupo 'sozinho' não é normal.\n"
     "• Homoscedasticidade: variâncias iguais (p > 0,05).\n"
     "• Médias: não há diferença significativa (p > 0,05)."),
    
    ("7) Conclusão",
     "Conclui-se que o fato de o estudante morar sozinho ou com outras pessoas "
     "não influencia significativamente o nível médio de dependência em smartphones. "
     "A hipótese nula de igualdade das médias é mantida (p = 0,2208).")
]

for title, content in sections:
    story.append(Paragraph(f"<b>{title}</b>", subtitle_style))
    story.append(Paragraph(content.replace("\n", "<br/>"), text_style))
    story.append(Spacer(1, 15))

# === Tabela resumo ===
data = [
    ["Teste", "Valor p", "Decisão (α = 0,05)"],
    ["Shapiro–Wilk (sozinho)", f"{p_sozinho:.4f}", "Não normal"],
    ["Shapiro–Wilk (acompanhado)", f"{p_outros:.4f}", "Normal"],
    ["Levene", f"{p_levene:.4f}", "Variâncias iguais"],
    ["t de Student", f"{p_t:.4f}", "Médias iguais"],
]
table = Table(data, colWidths=[190, 100, 200])
table.setStyle(TableStyle([
    ("BACKGROUND", (0, 0), (-1, 0), colors.lightgrey),
    ("GRID", (0, 0), (-1, -1), 0.5, colors.black),
    ("ALIGN", (1, 1), (-1, -1), "CENTER"),
]))
story.append(Spacer(1, 20))
story.append(Paragraph("<b>Tabela Resumo dos Resultados</b>", subtitle_style))
story.append(table)

# === Gerar PDF ===
doc.build(story)
print(f"PDF gerado com sucesso: {output_path}")


PDF gerado com sucesso: Atividade6_Grupo3.pdf
